<a href="https://colab.research.google.com/github/bradshimmin/llm_experiments/blob/main/OpenAI_Structured_Outputs_for_AI_Skills_tiktok_29aug24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Set up software
#
!pip -q install openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.2 MB/s eta 0:00:00


In [2]:
# Import libraries
#
import os
import openai
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import textwrap

from enum import Enum
from typing import Union
from pydantic import BaseModel
from openai import OpenAI

print("all done")

all done


In [3]:
# Load openai key
#
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OpenAI_key')

---
# Set up AI Skills data

In [4]:
# Mount Google Drive
#
from google.colab import drive
drive.mount('/content/gdrive')

print("all done")

Mounted at /content/gdrive
all done


In [5]:
# Load our sample dataset using 2q24 cleaned df
#
df_sample = pd.read_pickle("/content/gdrive/MyDrive/Colab Notebooks/Datasets/ai_skills_tracker/ai_skills_2q24/ai_skills_2q24_final_clean_28aug24.pkl")


In [6]:
df_sample.shape

(205517, 31)

In [7]:
df_sample.head(n=2)

,RecordID,Job title,Full job description,Date collected,Date posted,Week,Year,Month,Quarter,Hiring entity,...,Salary start,Salary end,Main industry,Job title category,AI technologies,AI skills,Source name,Source URL,File Date,Non-ASCII Chars
0,1,Data Engineer,"""ALL our jobs are US based and candidates must...",2023-06-11,2023-06-11,23,2023,6,2023Q2,TechFetch.com - On Demand Tech Workforce hirin...,...,NaN,NaN,IT Services,Statistician Data Science,"Bachelor, Ab Initio, Big Data, Healthcare Insu...","Databases, Computer Science, Information Techn...",LinkedIn,https://www.linkedin.com/jobs/view/data-engine...,2023-04-01,[]
1,2,Data Scientist,"""ALL our jobs are US based and candidates must...",2023-06-11,2023-06-11,23,2023,6,2023Q2,TechFetch.com - On Demand Tech Workforce hirin...,...,NaN,NaN,IT Services,Statistician AI,"Transformation, Sas, Azure, Databricks, Data L...","Support, Business Intelligence, Server, Langua...",LinkedIn,https://www.linkedin.com/jobs/view/data-scient...,2023-04-01,[]


In [8]:
# # prompt: Create a dataframe that only includes "Hiring entity" of "TikTok". Please ignore case
# #
# df_tiktok = df_sample[df_sample['Hiring entity'].str.lower() == 'tiktok']
# df_tiktok.shape


In [9]:
# df_tiktok_small = df_tiktok.sample(n=2).copy()
# df_tiktok_small.shape

In [10]:
# Pull out tiktok data
# Step 1: Filter by "Hiring entity" to include only "TikTok"
tiktok_jobs = df_sample[df_sample['Hiring entity'].str.contains("TikTok", case=False, na=False)]
tiktok_jobs.shape

(1640, 31)

In [11]:
# prompt: remove duplicates based on "Full job description"
#
tiktok_jobs = tiktok_jobs.drop_duplicates(subset=['Full job description'])
tiktok_jobs.shape

(1633, 31)

In [12]:

# Step 2: Convert "Date posted" to datetime format (modify the format if needed)
tiktok_jobs['Date posted'] = pd.to_datetime(tiktok_jobs['Date posted'], errors='coerce')

# Step 3: Filter jobs posted in the last 12 months
one_year_ago = datetime.now() - timedelta(days=547)
recent_tiktok_jobs = tiktok_jobs[tiktok_jobs['Date posted'] >= one_year_ago]

recent_tiktok_jobs.shape

(1060, 31)

In [13]:
recent_tiktok_jobs['Date posted'].max()

Timestamp('2024-06-17 00:00:00')

In [70]:
# Step 4: Randomly sample 100 jobs (or fewer if less than 100 are available)
sampled_jobs = recent_tiktok_jobs.sample(n=min(5, len(recent_tiktok_jobs)), random_state=42)

sampled_jobs.shape


(5, 31)

In [55]:
# prompt: Modify print() output so that the text wraps automatically at 250 charcters
def print_wrapped(text, width=100):
  wrapped_text = textwrap.fill(text, width=width)
  return wrapped_text


In [71]:
job_post = sampled_jobs['Full job description'].iloc[0]
print_wrapped(job_post)

'Responsibilities About TikTok U.S. Data SecurityTikTok is the leading destination for short-form\nmobile video. Our mission is to inspire creativity and bring joy. U.S. Data Security ("USDS") is a\nsubsidiary of TikTok in the U.S. This new, security-first division was created to bring heightened\nfocus and governance to our data protection policies and content assurance protocols to keep U.S.\nusers safe. Our focus is on providing oversight and protection of the TikTok platform and U.S. user\ndata, so millions of Americans can continue turning to TikTok to learn something new, earn a living,\nexpress themselves creatively, or be entertained. The teams within USDS that deliver on this\ncommitment daily span across Trust & Safety, Security & Privacy, Engineering, User & Product Ops,\nCorporate Functions and more.Why Join UsCreation is the core of TikTok\'s purpose. Our platform is\nbuilt to help imaginations thrive. This is doubly true of the teams that make TikTok possible.\nTogether, 

---
# Test OpenAI and Pydantic


In [76]:
# Set up a new, simplified Pydantic class
#
class Title(BaseModel):
    """
    Represents the full title of the job posting.

    This model captures the full title of the job posting
    Examples include:
    'Senior Machine Learning Engineer, TikTok Ads Core- Business Account',
    'Lead Research Engineer, Machine Learning - TikTok Privacy Innovation Lab'
    """
    title: str # Added type annotation for title

class Role(BaseModel):
    """
    Represents the area in which the employee will work.

    This model capture the area, department, or division in which the employee
    will work within the company. Examples include:
    'Research and development', 'User engagement', 'Systems infrastructure',
    'Trust and safety'
    """
    role: str # Added type annotation for role

class Objectives(BaseModel):
    """
    Represents the stated objectives the company has set for the employee.
    Examples include:
    ' 1. Drive insightful understanding of Live users and creators to accelerate
    better user, creator experience in Live.', '2. Keep track of core metrics and
    attribute metric changes to root causes.', '3. Use data and\nexperiments to
    verify hypotheses around bottlenecks in product growth, and consult on the
    direction for improvement.', '4. Work with MLE and product stakeholders to
    deLiver product insights and strategy.'
    """
    objectives: list[str] # Added type annotation for objectives

class Technologies(BaseModel):
    """
    Represents a technology-based skills required by a hiring entity.

    This model captures basic information about a products and technologies
    mentioned within this job post. Examples include:
    'Spark', 'Hadoop', 'PyTorch', 'Git', 'OpenAI', 'Vertex AI', 'Google Cloud',
    'Python', 'SQL', 'Llama', 'LangChain'
    """
    technologies: list[str]  # Added type annotation for technologies

class Skills(BaseModel):
    """
    Represents a technology-based skills required by a hiring entity.

    This model captures basic information about a business skills mentioned
    within this job post. Examples include:
    'data collection', 'data cleaning', 'data analysis', 'reporting',
    'data visualization', 'training', 'stakeholder collaboration', 'MLOps',
    'DataOps', 'Responsible AI', 'Data Engineering', 'Data Science', 'Data Warehousing'
    """
    skills: list[str]  # Added type annotation for skills

class Summary(BaseModel):
    """
    Represents the area in which the employee will work.

    This model capture the a concise, 250 word summary of the job posting, focusing
    on the job itself rather than the company or the job requirements and benefits.'
    """
    summary: str # Added type annotation for role

class JobResponseSimple(BaseModel):
    """
    Represents the structured response for an article analysis.

    This model aggregates information about title, role, objectives, technologies,
    and skills mentioned in a job post
    """
    title: str
    role: str
    objectives: list[Objectives]
    technologies: list[Technologies]
    skills: list[Skills]
    summary: str

In [67]:
# Simplify our inference function
#
client = OpenAI()

title = ""
role = ""
objectives = []
technologies = []
skills = []
summary = ""

def parse_post_simple(jobpost):
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
            {"role": "system", "content": "You are a helpful assistant that \
            scans job posts for organization info, the technologies and products, \
            and business skills required for a given job posting. Please be thorough \
            and include as many technologies and skills as you can in your response. \
            Do not include the examples UNLESS they actual appear in the jobpost."},
            {"role": "user", "content": jobpost},
        ],
        response_format=JobResponseSimple,
    )

    message = completion.choices[0].message
    if message.parsed:
        output = []
        output.append(str(message.parsed.title))
        output.append(str(message.parsed.role))
        for objectives in message.parsed.objectives:
            output.append(str(objectives))
        for technologies in message.parsed.technologies:
            output.append(str(technologies))
        for skills in message.parsed.skills:
            output.append(str(skills))
        output.append(str(message.parsed.summary))
        return output
    else:
        return message.refusal

In [68]:
# Run our test
#
test_ner = parse_post_simple(job_post)
print(test_ner)

['Analyst - U.S. Data Security', 'Analyst', "objectives=['Apply investigational and analytic skills to monitor, audit, and analyze potential fraudulent cases and consolidate anti-fraud rules', 'Design processes and build effective workflows to define characteristics of different fraud types and mitigate fraud', 'Define key performance metrics and build detailed reporting models for effective tracking and reporting on measures implemented', 'Research fraud-related policies and build effective enforcement processes', 'Communicate findings and complex results to key stakeholders and manage timeline of key projects']", "technologies=['SQL', 'Python', 'R']", "skills=['Quantitative analysis', 'Data analysis', 'Fraud management', 'Project management', 'Collaboration', 'Reporting', 'Statistical analysis']", "The Analyst role at TikTok's U.S. Data Security team involves managing large-scale quantitative analysis projects, identifying and addressing fraudulent activities, and collaborating with 

---
# Append data to individual columns in our dataframe


In [78]:
# Inference function definition
#
title = ""
role = ""
objectives = []
technologies = []
skills = []
summary = ""

def parse_post_simple(jobpost):
    completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
            {"role": "system", "content": "You are a helpful assistant that \
            scans job posts for organization info, the technologies and products, \
            and business skills required for a given job posting. Please be thorough \
            and include as many technologies and skills as you can in your response. \
            Do not include the examples UNLESS they actual appear in the jobpost."},
            {"role": "user", "content": jobpost},
        ],
        response_format=JobResponseSimple,
    )

    message = completion.choices[0].message
    if message.parsed:
        # Separate lists for technologies and business skills
        title = str(message.parsed.title)
        role = str(message.parsed.role)
        objectives = [str(obj) for obj in message.parsed.objectives]
        technologies = [str(tech) for tech in message.parsed.technologies]
        skills = [str(skill) for skill in message.parsed.skills]
        summary = str(message.parsed.summary)

        # Return as separate values
        return title, role, objectives, technologies, skills, summary
    else:
        return "", "", [], [], [], ""  # Return empty lists in case of refusal or no parsing


In [77]:
# Convert sample_jobs into df for processing
#
df = sampled_jobs.copy()
df.shape

(5, 31)

In [79]:
# Apply the parse_post_simple function to each job post
#
df['title'], df['role'], df['objectives'], df['technologies'], df['skills'], df['summary'] \
= zip(*sampled_jobs['Full job description'].apply(parse_post_simple))


In [88]:
df.columns

Index(['RecordID', 'Job title', 'Full job description', 'Date collected',
       'Date posted', 'Week', 'Year', 'Month', 'Quarter', 'Hiring entity',
       'Entity size', 'Home office hybrid work', 'Full or part time status',
       'Number of applicants', 'Longitude', 'Latitude', 'City', 'Territory',
       'Region', 'Experience level', 'Education level', 'Salary start',
       'Salary end', 'Main industry', 'Source name', 'Source URL', 'File Date',
       'title', 'role', 'objectives', 'technologies', 'skills', 'summary'],
      dtype='object')

In [97]:
# Check results
#
def check_results(df, record):
    """Check the results of the parse_post_simple function."""
    print(f"title: {df.title.iloc[record]}")
    print(f"role: {df.role.iloc[record]}")
    print(f"objectives: {df.objectives.iloc[record]}")
    print(f"technologies: {df.technologies.iloc[record]}")
    print(f"skills: {df.skills.iloc[record]}")
    print(f"summary: {df.summary.iloc[record]}")


In [98]:
check_results(df, 2)


title: Data Analyst
role: Data Team Member
objectives: ["objectives=['Improve the recommendation strategy of short video service.', 'Understand user behavior in depth and optimize user experience such as creation and browsing.', 'Optimize the index system of recommendation strategy and do attribution analysis on abnormal indicators.', 'Analyze business data in different markets, formulate targeted recommendation strategies, and evaluate strategic benefits.', 'Study special topics such as ecological growth of short video content.']"]
technologies: ["technologies=['machine learning', 'recommendation algorithms', 'data analysis']"]
skills: ["skills=['user empathy', 'problem solving', 'analytical skills', 'communication skills', 'logical thinking', 'data analysis of Internet companies']"]
summary: The role involves improving recommendation strategies for TikTok's short video service, understanding user behavior, and optimizing user experience through data analysis.


In [82]:
# Drop 'unwanted columns
#
df = df.drop(['Job title category', 'AI technologies', 'AI skills', 'Non-ASCII Chars'], axis=1)
df.columns

Index(['RecordID', 'Job title', 'Full job description', 'Date collected',
       'Date posted', 'Week', 'Year', 'Month', 'Quarter', 'Hiring entity',
       'Entity size', 'Home office hybrid work', 'Full or part time status',
       'Number of applicants', 'Longitude', 'Latitude', 'City', 'Territory',
       'Region', 'Experience level', 'Education level', 'Salary start',
       'Salary end', 'Main industry', 'Source name', 'Source URL', 'File Date',
       'title', 'role', 'objectives', 'technologies', 'skills', 'summary'],
      dtype='object')

In [83]:
# Write our new df to disk
#
df.to_pickle("/content/gdrive/MyDrive/Colab Notebooks/Datasets/df_ner_tiktok.pkl")
df.to_csv("/content/gdrive/MyDrive/Colab Notebooks/Datasets/df_ner_tiktok.csv")

---
# Summarizing and analyzing our full list of jobs

In [101]:
# Combine our test data into a single block of text
#
job_data = "\n\n".join([
    f"Title: {row['title']}\n"
    f"Date posted: {row['Date posted']}\n"
    f"Role: {row['role']}\n"
    f"Objectives: {row['objectives']}\n"
    f"Technologies: {row['technologies']}\n"
    f"Skills: {row['skills']}\n"
    f"Summary: {row['summary']}\n"
    for index, row in df.iterrows()
])
print(job_data)

Title: Data Analyst - U.S. Data Security
Date posted: 2024-04-24 00:00:00
Role: Data Analyst
Objectives: ["objectives=['Manage end-to-end large-scale quantitative analysis projects and monitoring systems.', 'Audit and analyze potential fraudulent cases and consolidate anti-fraud rules.', 'Design processes and effective workflows to define characteristics of different fraud types and capture new risk types.', 'Collaborate with cross-functional teams to drive improvements in fraud management tools and processes.']", "objectives=['Design monitoring tools to mitigate fraud based on trends in local markets.', 'Define key performance metrics and build detailed reporting models for tracking and feedback.', 'Research fraud-related policies and build enforcement processes to prevent and penalize bad actors.']", "objectives=['Communicate complex results to key stakeholders and manage project timelines.', 'Supervise analysts and provide guidance on fraud policies and enforcement measures.']"]
Tec

In [110]:
# Generate prompt with job data
#
system_prompt = f"""
You are an AI assistant specializing in analyzing job data to understand a company's strategic focus and workforce priorities.
Given multiple job postings with roles, objectives, technologies, skills, and summaries, your task is to generate a comprehensive markdown summary
detailing the overall nature of the job roles and what the company is focusing on. Use the following outline to structure the analysis:

1. **Introduction**
   Provide an introduction about TikTok’s current focus and strategic areas based on the job data analyzed.

2. **Job Roles and Objectives**
   Summarize the main job roles within the data set, detailing the key objectives of these roles. Highlight common themes and areas of strategic emphasis.

3. **Technologies and Tools**
   Identify the technologies and tools most frequently mentioned across the job posts. Explain how these technologies contribute to TikTok’s overall strategy.

4. **Skills in Demand**
   Outline the critical skills that TikTok seeks in its employees, both technical and soft skills. Discuss how these skills align with the company’s objectives.

5. **Strategic Focus Areas**
   Discuss what the job data suggests about TikTok’s strategic focus.

6. **Conclusion**
   Provide a conclusion summarizing TikTok’s overarching goals based on the job data, highlighting any emerging trends or notable observations.

**Output Format**: Provide a markdown-formatted analysis based on the data.
"""


In [132]:
# Test out our analysis
#
def generate_analysis(system_prompt, job_data):
    """Generate analysis based on job data."""
    job_data = str(job_data)

    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini", # or we could try gpt-4o
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": system_prompt
                }
            ]
            },
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": "**Data Input for Analysis:**" + job_data
                }
            ]
            },
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={
            "type": "text"
        }
    )
    print(response.choices[0].message.content)

In [133]:
# Print out our results
#
generate_analysis(system_prompt, job_data)


# TikTok Job Data Analysis

## Introduction
This analysis examines TikTok's job postings to ascertain the company’s current strategic focus and workforce priorities. The data illustrates a strong emphasis on data security, machine learning systems, and the enhancement of user experience through data analysis and recommendation algorithms. TikTok is clearly positioning itself at the forefront of technology and innovation, particularly in the realms of machine learning, privacy, and data analytics.

## Job Roles and Objectives
The job roles identified within the analyzed postings reflect a blend of analytical, technical, and research-focused positions. The key roles include:

1. **Data Analyst (U.S. Data Security)**: Focuses on managing large-scale data analysis to detect and mitigate fraudulent activities, collaborating with teams to enhance fraud management processes.
  
2. **Machine Learning Systems Engineer**: Tasked with developing and optimizing machine learning infrastructures, pa